# 2023-01-30 self attention layers

In [1]:
# download data

In [2]:
import torch
torch.cuda.device_count(), torch.cuda.current_device()

(1, 0)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
# read on review data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# Here is all unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [7]:
# create a mapping from characters to integers
ctoi = {c: i for i, c in enumerate(chars)}
itoc = {i: c for i, c in enumerate(chars)}

encode = lambda s: [ctoi[c] for c in s]
decode = lambda l: ''.join([itoc[i] for i in l])

print(encode("hi there"))

print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [8]:
# Let now encode the entire text dataset and store it into torch.Tensor
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

data[0:100]

torch.Size([1115394]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [9]:
# Let's now split up the data into train set and validation set
n = round(len(data) * 0.9);
train_data = data[:n]
val_data   = data[n:]
len(train_data), len(val_data)

(1003855, 111539)

In [10]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when context is {context} the target: {target}.")

when context is tensor([18]) the target: 47.
when context is tensor([18, 47]) the target: 56.
when context is tensor([18, 47, 56]) the target: 57.
when context is tensor([18, 47, 56, 57]) the target: 58.
when context is tensor([18, 47, 56, 57, 58]) the target: 1.
when context is tensor([18, 47, 56, 57, 58,  1]) the target: 15.
when context is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47.
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58.


In [12]:
torch.manual_seed(1337)

batch_size = 4
block_size = 8

def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(train_data) - block_size, (batch_size, ))
    xb = torch.stack([train_data[i:i+block_size] for i in ix])
    yb = torch.stack([train_data[i+1:i+1+block_size] for i in ix])
    xb, yb = xb.to(device), yb.to(device)
    return xb, yb

xb, yb = get_batch()
print(xb)
print(yb)
for b in range(batch_size):
    for i in range(block_size):
        x = xb[b,:i+1]
        y = yb[b,i]
        print(f"when context is {x} the target: {y}.")

tensor([[56,  6,  0, 24, 43, 58,  1, 61],
        [39, 47, 51,  1, 58, 46, 39, 58],
        [52, 45,  1, 58, 53,  1, 57, 39],
        [43, 47, 52, 45,  1, 46, 53, 50]], device='cuda:0')
tensor([[ 6,  0, 24, 43, 58,  1, 61, 46],
        [47, 51,  1, 58, 46, 39, 58,  1],
        [45,  1, 58, 53,  1, 57, 39, 63],
        [47, 52, 45,  1, 46, 53, 50, 47]], device='cuda:0')
when context is tensor([56], device='cuda:0') the target: 6.
when context is tensor([56,  6], device='cuda:0') the target: 0.
when context is tensor([56,  6,  0], device='cuda:0') the target: 24.
when context is tensor([56,  6,  0, 24], device='cuda:0') the target: 43.
when context is tensor([56,  6,  0, 24, 43], device='cuda:0') the target: 58.
when context is tensor([56,  6,  0, 24, 43, 58], device='cuda:0') the target: 1.
when context is tensor([56,  6,  0, 24, 43, 58,  1], device='cuda:0') the target: 61.
when context is tensor([56,  6,  0, 24, 43, 58,  1, 61], device='cuda:0') the target: 46.
when context is tensor(

In [13]:
print(xb) # input to our transformer

tensor([[56,  6,  0, 24, 43, 58,  1, 61],
        [39, 47, 51,  1, 58, 46, 39, 58],
        [52, 45,  1, 58, 53,  1, 57, 39],
        [43, 47, 52, 45,  1, 46, 53, 50]], device='cuda:0')


In [14]:
# self attention layer

In [15]:
import torch

torch.manual_seed(1338)

B, T, C = 4, 8, 2 ; # batch, time, channel

x = torch.randn((B, T, C))

x[0]

tensor([[-1.3113, -1.0017],
        [-1.2342,  0.1297],
        [-0.5150, -1.2666],
        [-0.6719,  0.1851],
        [ 0.9367,  0.3139],
        [-1.3950,  0.1132],
        [ 0.3622,  2.5192],
        [-0.7672, -0.9529]])

In [16]:
# version 1
xbow = torch.zeros_like(x)
for b in range(B):
    for t in range(T):
        x_prev = x[b, :t+1] # t, C
        xbow[b,t] = torch.mean(x_prev, dim=0)
xbow[0]

tensor([[-1.3113, -1.0017],
        [-1.2728, -0.4360],
        [-1.0202, -0.7129],
        [-0.9331, -0.4884],
        [-0.5591, -0.3279],
        [-0.6985, -0.2544],
        [-0.5469,  0.1418],
        [-0.5745,  0.0050]])

In [17]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # (B, T, T) x (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [18]:
# version 3
# using solfmax
wei = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(wei==0, -torch.inf)
wei = torch.softmax(wei, dim=-1)
xbow3 = wei @ x # (B, T, T) x (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow3)

True

In [19]:
import torch
import torch.nn as nn

torch.manual_seed(1337)

B, T, C = 4, 8, 32 ; # batch, time, channel
head_size = 16

x = torch.randn((B, T, C))

query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

q = query(x) # (B, T, head_size)
k = key(x) # (B, T, head_size)
v = value(x) # (B, T, head_size)

wei = q@k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, head_size, T) -> (B, T, T)
wei = wei / torch.sqrt(torch.tensor(head_size))
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, -torch.inf)
wei = torch.softmax(wei, dim=-1)

out = wei @ v # (B, T, T) x (B, T, C) -> (B, T, C)

q.shape, k.shape, v.shape, wei.shape, out.shape

(torch.Size([4, 8, 16]),
 torch.Size([4, 8, 16]),
 torch.Size([4, 8, 16]),
 torch.Size([4, 8, 8]),
 torch.Size([4, 8, 16]))

In [21]:
# create a class for self attention layer
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(C, head_size)
        self.key = nn.Linear(C, head_size)
        self.value = nn.Linear(C, head_size)
        self.register_buffer('tril', torch.tril(torch.ones(T, T)))

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # B, T, head_size
        k = self.key(x) # B, T, head_size
        v = self.value(x) # B, T, head_size
        # computer attention score
        wei = q @ v.transpose(-2, -1) * head_size ** -0.5 # (B, T, head_size) x (B, head_size, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.softmax(wei,dim=-1) # (B, T, T)
        # perform the weighted aggregation
        out = wei@v # (B, T, T) x (B, T, head_size) -> (B, T, head_size)
        return out
    
class MultipleHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        out = torch.cat([head(x) for head in self.heads], -1)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
                        nn.Linear(n_embd, n_embd),
                        nn.ReLU())
    
    def forward(self, x):
        x = self.net(x)
        return x
    
class Block(nn.Module):
    def __init__(self, num_heads, n_embd):
        super().__init__()
        head_size = n_embd//num_heads
        self.sa_head = MultipleHeadAttention(num_heads, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = self.sa_head(self.ln1(x)) + x
        x = self.ffwd(self.ln2(x)) + x
        return x

head = Head(2)
out = head(x)
print(out.shape)


multi_heads = MultipleHeadAttention(4, 2)
out = multi_heads(x)
print(out.shape)

ffwd = FeedForward(8)
out = ffwd(out)
print(out.shape)

x = torch.randn((4, 8, 32))
block = Block(4, 32)
out = block(x)
print(out.shape)

blocks = nn.Sequential(*nn.ModuleList([Block(4, 32) for _ in range(4)]))
blocks(x).shape

torch.Size([4, 8, 2])
torch.Size([4, 8, 8])
torch.Size([4, 8, 8])
torch.Size([4, 8, 32])


torch.Size([4, 8, 32])

In [ ]:
torch.manual_seed(1338)
x = torch.randn((4, 8, 32))
print(x[0, 0].mean(), x[0, 0].var())

layer_norm = nn.LayerNorm(32)
x = layer_norm(x)
print(x[1, 1].mean(), x[2, 2].var())

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)

# bigram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(T, n_embd)
        self.blocks = nn.Sequential(*nn.ModuleList([Block(4, 32) for _ in range(4)] + [nn.LayerNorm(n_embd)]))
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb # (B, T, n_embd)
        x = self.blocks(x)
        logits = self.lm_head(x) # (B, n_embd, vocab_size) x (B, T, n_embd) -> (B, T, vocab_size)
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return loss, logits

    @torch.no_grad()
    def generate(self, idx, max_new_token):
        for i in range(max_new_token):
            loss, logits = self(idx[:, -block_size:])
            logits = logits[:, -1,:]
            probs = F.softmax(logits, -1)
            next_idx = torch.multinomial(probs, 1)
            idx = torch.cat([idx, next_idx], 1)
        return idx
    
B, T, C = 32, 8, 32 ; # batch, time, channel
n_embd = 32
batch_size, block_size = B, T
max_iter = 3000
num_heads = 4
num_blocks = 4
eval_iters = 200
eval_interval = 300

head_size = 16
xb, yb = get_batch()
print(xb.shape, yb.shape)

model = BigramLanguageModel(vocab_size).to(device)
m = model.to(device)

loss, logits = model(xb, yb)
print(loss)

torch.Size([32, 8]) torch.Size([32, 8])
tensor(4.3111, device='cuda:0', grad_fn=<NllLossBackward0>)


In [29]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    losses = torch.zeros(eval_iters)
    out = {}
    for split in ['train', 'val']:
        for i in range(eval_iters):
            xb, yb = get_batch()
            loss, logits = model(xb, yb)
            losses[i] = loss
        out[split] = losses.mean().item()
    return out

In [30]:
sent = model.generate(idx=torch.zeros((1,1), dtype=torch.long).to(device), max_new_token=1000)
print(decode(sent[0].tolist()))


?qf;xBDkRZkNwc'wj,ZT,OLXL,eHtK
b:!GjwkMBbzA$3:XaSGgO-3nSMGj?la3auhX:YVXGthXMNuwqhBM$G.t? w dXlNDwaAen?w3cHPwRWf,GDnZnYzxzqImQX
Yow&$LMtUfCiGIqBi!&V!$W;KwilN,w
e3 irweY?vnxcin;laW;HFGEZroG EsSXUB;qWk GnYGw3.FYWjbm!UelJlinFAU; w.C-nxD3qcnc XbN:'Uup;Mnis'X3Uwty;H:'nBnUHIkGBmTpjY-ggvIEjMP:D:lqwYdqmBtSkklmoaW-nNAXQhjGCeIib3q wyS;G&dM$HZqETxiGGhiq$Aw  -LKKgAe-;'3H
QQpkNMmnunrup!!A s
;;3;QDLWnm:f'GwCey$nlMUE$tw,NMfMPrD?UXY'S? .$rHK-NLbk!r3,wyb&i-:
 adUaUWG$3YE'zFHYBiuih KN.k?DUraAnyHRrG:tMmc&fn VE?!NMJkenN GnMPk3RmGY..AYBXq'WG3GwUS?m !bBSB;e$hsN.-wXn;mnbBqqrxmiYArXaJRY;$$Oz;BjWUHGwtn qqugUbxIE.T$
mMc$  YX:y:BsSID&$Ywgl.I AnEnMoAGJUlMpKwjjkXxsYuzXjPTAbnVlMovMvL-Z?ACa3!UL.XM'esF'Mp-vUMKH.iew:RSu
q A$GxVM'eRzYXALU:y,D$tGCCI!;sMPknYikYUSn3iQhMUeUn,FXrxmgqjGhZcnrhb;akwtOGXwvwB-E.
'ww3m k ! .vDCBUml nFm,CMZ!QNwZlN nR'MG iyBOYF'WLLIiGZgA?NXImiwu
bMp$
&XnGen,QzXCe?tBy
XeHBIYXHoi3MiuGoKtneCniLZn e A3mmn xRpDrtG:'x wMb:A$ngnYKD M$zkw-3EYBlWQlbkm, oKUrSYMO
F!r;HMb.ZTBwrtI,&LivY$WYZtTi, n;!u;qy-CkWwgioS

In [31]:
# create pytorch optimizer
from torch.optim import AdamW
import numpy as np


optimizer = AdamW(model.parameters())

In [32]:
# do optimize
def train():
    for i in range(max_iter):
        if i % eval_iters == 0:
            out = estimate_loss()
            print(f"Train loss: {out['train']}. Val loss: {out['val']}. ")
        xb, yb = get_batch()
        loss, logits = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

train()

sent = model.generate(idx=torch.zeros((1,1), dtype=torch.long).to(device), max_new_token=500)
print(decode(sent[0].tolist()))

Train loss: 4.333967208862305. Val loss: 4.338881015777588. 
Train loss: 2.772444486618042. Val loss: 2.7778167724609375. 
Train loss: 2.5268447399139404. Val loss: 2.5290567874908447. 
Train loss: 2.4437453746795654. Val loss: 2.4444539546966553. 
Train loss: 2.3825788497924805. Val loss: 2.3829026222229004. 
Train loss: 2.354346513748169. Val loss: 2.34036922454834. 
Train loss: 2.324941635131836. Val loss: 2.332179546356201. 
Train loss: 2.2876574993133545. Val loss: 2.292999267578125. 
Train loss: 2.287186622619629. Val loss: 2.26741099357605. 
Train loss: 2.2644283771514893. Val loss: 2.253140687942505. 
Train loss: 2.2381105422973633. Val loss: 2.2442944049835205. 
Train loss: 2.2095961570739746. Val loss: 2.2272608280181885. 
Train loss: 2.209355115890503. Val loss: 2.207448959350586. 
Train loss: 2.2141804695129395. Val loss: 2.201678514480591. 
Train loss: 2.1920230388641357. Val loss: 2.197150468826294. 

MTIO:
Louct wed.

Foir cet Henk whe wher the ine, heer to tano tono to 

In [33]:
train()
    
sent = model.generate(idx=torch.zeros((1,1), dtype=torch.long).to(device), max_new_token=500)
print(decode(sent[0].tolist()))

Train loss: 2.1779205799102783. Val loss: 2.1692137718200684. 
Train loss: 2.1849870681762695. Val loss: 2.164435863494873. 
Train loss: 2.173689126968384. Val loss: 2.162363290786743. 
Train loss: 2.1449947357177734. Val loss: 2.164212703704834. 
Train loss: 2.1526119709014893. Val loss: 2.1528480052948. 
Train loss: 2.1438169479370117. Val loss: 2.1391239166259766. 
Train loss: 2.1338107585906982. Val loss: 2.13897705078125. 
Train loss: 2.1347973346710205. Val loss: 2.129775047302246. 
Train loss: 2.123077154159546. Val loss: 2.1253042221069336. 
Train loss: 2.1180872917175293. Val loss: 2.1241564750671387. 
Train loss: 2.1050477027893066. Val loss: 2.1105117797851562. 
Train loss: 2.0973353385925293. Val loss: 2.098381280899048. 
Train loss: 2.10115122795105. Val loss: 2.108600378036499. 
Train loss: 2.1069040298461914. Val loss: 2.0965824127197266. 
Train loss: 2.100752115249634. Val loss: 2.085097312927246. 

Thins griook wis;
Hor pedcrars, evas;
The but of a-fatheree:
On!
Broutt

### Convert this file to md

In [ ]:
from IPython.core.display import Javascript

In [ ]:
%%js
IPython.notebook.kernel.execute('this_notebook = "' + IPython.notebook.notebook_name + '"')

In [ ]:
this_notebook

In [ ]:
!jupyter nbconvert --to markdown {this_notebook} --output-dir=../_posts